# Imports

In [99]:
import pandas as pd
import os
import glob

# Constants

In [100]:
frequency = 100 # samples per sec
cut_time = 3 # seconds
cut_amount = frequency * cut_time

In [101]:
features = [
    'attitude.roll', 'attitude.pitch', 'attitude.yaw',
    'gravity.x', 'gravity.y', 'gravity.z',
    'rotationRate.x', 'rotationRate.y', 'rotationRate.z',
    'userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z'
]

labels_dict = {'wlk': 0, 'sit': 1, "std": 2, "ups": 3, "jog": 4, "dws": 5}
labels = list(labels_dict.keys())

In [102]:
cwd = os.getcwd()
data_dir = os.path.join(cwd, 'real-data')

In [103]:
partc = 100

# Helpers

In [104]:
def cut_edges(df, amount):
    return df[amount:-amount]

In [105]:
def drop_columns(df, features):
    columns = list(df.columns)
    cols_to_drop = [col for col in columns if col not in features]
    return df.drop(cols_to_drop, axis=1)

In [106]:
def clean_df(df, amount, features):
    new_df = cut_edges(df, amount)
    new_df = drop_columns(new_df, features)
    return new_df

In [107]:
def label_file_names(label, filedir):
    return glob.glob(filedir + '/' + label + '*.csv')

# Process Data

In [109]:
frames = []
for label in labels:
    files = label_file_names(label, data_dir)
    for file in files:
        df = pd.read_csv(file)
        df = clean_df(df, cut_amount, features)
        df['partc'] = partc
        df['action'] = label
        df['action_file_index'] = labels_dict.get(label)
        frames.append(df)
        print('added file: ' + file)
full_data = pd.concat(frames)

added file: /Users/Ido/workshop/notebooks/real-data/sit3.csv
added file: /Users/Ido/workshop/notebooks/real-data/sit2.csv
added file: /Users/Ido/workshop/notebooks/real-data/sit1.csv
added file: /Users/Ido/workshop/notebooks/real-data/std2.csv
added file: /Users/Ido/workshop/notebooks/real-data/std3.csv
added file: /Users/Ido/workshop/notebooks/real-data/std1.csv
added file: /Users/Ido/workshop/notebooks/real-data/ups2.csv
added file: /Users/Ido/workshop/notebooks/real-data/ups3.csv
added file: /Users/Ido/workshop/notebooks/real-data/ups1.csv
added file: /Users/Ido/workshop/notebooks/real-data/ups4.csv
added file: /Users/Ido/workshop/notebooks/real-data/jog2.csv
added file: /Users/Ido/workshop/notebooks/real-data/jog3.csv
added file: /Users/Ido/workshop/notebooks/real-data/jog1.csv
added file: /Users/Ido/workshop/notebooks/real-data/jog4.csv
added file: /Users/Ido/workshop/notebooks/real-data/dws3.csv
added file: /Users/Ido/workshop/notebooks/real-data/dws2.csv
added file: /Users/Ido/w

### Compress result to gzip

In [110]:
full_data.to_csv('full_real_data.gz', compression='gzip')